In [2]:
import spacy
from nltk.tokenize import sent_tokenize
import nltk

# Garantindo que o tokenizador de sentenças está baixado
try:
    nltk.data.find('tokenizers/punkt_tab')
except LookupError:
    nltk.download('punkt_tab')

# Carregando SpaCy
try:
    nlp = spacy.load("pt_core_news_sm")
except:
    print("Modelo SpaCy não encontrado.")

# Configuração (Dicionários)
palavras_positivas = ["bom", "excelente", "ótimo", "amar", "gostar", "rápido", "maravilhoso", "confiável"]
palavras_negativas = ["ruim", "péssimo", "horrível", "odiar", "lento", "caro", "quebrado", "feio", "triste", "confuso"]

def analisar_por_sentenca(caminho_arquivo):
    try:
        with open(caminho_arquivo, "r", encoding="utf-8") as f:
            texto_completo = f.read()
            
        # 1. QUEBRA EM SENTENÇAS
        sentencas = sent_tokenize(texto_completo, language='portuguese')
        
        placar_final = {"Positivo": 0, "Negativo": 0, "Neutro": 0}
        
        print(f"--- Analisando {len(sentencas)} sentenças... ---")
        
        # 2. ANÁLISE INDIVIDUAL (Loop frase a frase)
        for frase in sentencas:
            doc = nlp(frase.lower())
            tokens = [token.lemma_ for token in doc]
            
            pontos_pos = 0
            pontos_neg = 0
            
            for token in tokens:
                if token in palavras_positivas:
                    pontos_pos += 1
                elif token in palavras_negativas:
                    pontos_neg += 1
            
            if pontos_pos > pontos_neg:
                placar_final["Positivo"] += 1
            elif pontos_neg > pontos_pos:
                placar_final["Negativo"] += 1
            else:
                placar_final["Neutro"] += 1

        # contagem
        total_votos = placar_final["Positivo"] + placar_final["Negativo"]
        
        if total_votos == 0:
            return "Neutro (Sem dados suficientes)", 0.0, placar_final

        if placar_final["Positivo"] > placar_final["Negativo"]:
            resultado = "Positivo"
            probabilidade = placar_final["Positivo"] / total_votos
        elif placar_final["Negativo"] > placar_final["Positivo"]:
            resultado = "Negativo"
            probabilidade = placar_final["Negativo"] / total_votos
        else:
            resultado = "Neutro (Empate de Sentenças)"
            probabilidade = 0.5
            
        return resultado, probabilidade, placar_final

    except FileNotFoundError:
        return "Erro: Arquivo não encontrado", 0.0, {}

# --- EXECUÇÃO ---
arquivo_alvo = "dataset_reviews_random.txt" 

res, prob, placar = analisar_por_sentenca(arquivo_alvo)

print(f"\nResultado Final do Documento: {res}")
print(f"Confiança (Baseada em frases): {prob:.2%}")
print(f"Placar Detalhado: {placar}")

--- Analisando 5000 sentenças... ---

Resultado Final do Documento: Positivo
Confiança (Baseada em frases): 53.14%
Placar Detalhado: {'Positivo': 1665, 'Negativo': 1468, 'Neutro': 1867}
